In [5]:
# Question 2 (car_evaluation)

In [3]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
# part a (dataset import and preprocessing)
df = pd.read_csv('car_evaluation.csv')
df.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [7]:
df.isnull().sum()

,0
vhigh,0
vhigh.1,0
2,0
2.1,0
small,0
low,0
unacc,0


In [10]:
df.describe()

,vhigh,vhigh.1,2,2.1,small,low,unacc
count,1727,1727,1727,1727,1727,1727,1727
unique,4,4,4,3,3,3,4
top,high,high,3,4,med,med,unacc
freq,432,432,432,576,576,576,1209


In [16]:
df.shape

(1727, 7)

In [19]:
# part b (encoding categorical variables into ordinal or binary numbers)

import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# 1. Load the dataset and assign column names
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv("car_evaluation.csv", header=None, names=column_names)

# 2. Define Ordinal Encoding Mappings (defining the correct order for the encoder)
# Order: low=0, med=1, high=2, vhigh=3
order_high_low = ['low', 'med', 'high', 'vhigh']
# Order: small=0, med=1, big=2
order_size = ['small', 'med', 'big']
# Order: low=0, med=1, high=2
order_safety = ['low', 'med', 'high']
# Order: 2=0, 3=1, 4=2, 5more=3
order_doors = ['2', '3', '4', '5more']
# Order: 2=0, 4=1, more=2
order_persons = ['2', '4', 'more']
# Target Order: unacc=0, acc=1, good=2, vgood=3
order_class = ['unacc', 'acc', 'good', 'vgood']

# Full list of categories for the OrdinalEncoder (must match column order)
categories = [
    order_high_low, # buying
    order_high_low, # maint
    order_doors,    # doors
    order_persons,  # persons
    order_size,     # lug_boot
    order_safety,   # safety
    order_class     # class (target)
]

# 3. Initialize and fit OrdinalEncoder
encoder = OrdinalEncoder(categories=categories)
df_encoded = pd.DataFrame(encoder.fit_transform(df), columns=column_names)

# 4. Display the encoded data
print(df_encoded.head())

# 5. Display the encoding scheme for easy reference
print("\n--- Encoding Scheme ---")
print(f"buying/maint: {order_high_low} -> [0, 1, 2, 3]")
print(f"doors: {order_doors} -> [0, 1, 2, 3]")
print(f"persons: {order_persons} -> [0, 1, 2]")
print(f"lug_boot: {order_size} -> [0, 1, 2]")
print(f"safety: {order_safety} -> [0, 1, 2]")
print(f"class (Target): {order_class} -> [0, 1, 2, 3]")

   buying  maint  doors  persons  lug_boot  safety  class
0     3.0    3.0    0.0      0.0       0.0     0.0    0.0
1     3.0    3.0    0.0      0.0       0.0     1.0    0.0
2     3.0    3.0    0.0      0.0       0.0     2.0    0.0
3     3.0    3.0    0.0      0.0       1.0     0.0    0.0
4     3.0    3.0    0.0      0.0       1.0     1.0    0.0

--- Encoding Scheme ---
buying/maint: ['low', 'med', 'high', 'vhigh'] -> [0, 1, 2, 3]
doors: ['2', '3', '4', '5more'] -> [0, 1, 2, 3]
persons: ['2', '4', 'more'] -> [0, 1, 2]
lug_boot: ['small', 'med', 'big'] -> [0, 1, 2]
safety: ['low', 'med', 'high'] -> [0, 1, 2]
class (Target): ['unacc', 'acc', 'good', 'vgood'] -> [0, 1, 2, 3]


In [29]:
# part c (decision tree classifier)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

# Define the column names for the car evaluation dataset
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

# --- Data Reload and Encoding (from previous steps) ---
df = pd.read_csv("car_evaluation.csv", header=None, names=column_names)

# Define the necessary ordinal orderings
order_high_low = ['low', 'med', 'high', 'vhigh']
order_size = ['small', 'med', 'big']
order_safety = ['low', 'med', 'high']
order_doors = ['2', '3', '4', '5more']
order_persons = ['2', '4', 'more']
order_class = ['unacc', 'acc', 'good', 'vgood']

categories = [order_high_low, order_high_low, order_doors, order_persons, order_size, order_safety, order_class]
encoder = OrdinalEncoder(categories=categories)
df_encoded = pd.DataFrame(encoder.fit_transform(df), columns=column_names)

# 1. Prepare Features (X) and Target (y)
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

# 2. Split the data (70% Train, 30% Test, stratified for class imbalance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 3. Import and Build the Decision Tree Classifier
# Set criterion='gini' and max_depth=3
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
clf_gini.fit(X_train, y_train)

# 4. Predict the Test set results
y_pred_test = clf_gini.predict(X_test)

# 5. Calculate and show the accuracy score
test_accuracy = accuracy_score(y_test, y_pred_test)

print("--- Decision Tree Classification Results (Gini, Max Depth=3) ---")
print(f"Test Set Accuracy Score: {test_accuracy:.4f}")
print("---------------------------------------------------------------")

--- Decision Tree Classification Results (Gini, Max Depth=3) ---
Test Set Accuracy Score: 0.7784
---------------------------------------------------------------


In [30]:
# part d ( comparison of test and training set)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

# --- 1. Data Reload and Encoding ---
# (Necessary to ensure X_train and y_train are defined)
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv("car_evaluation.csv", header=None, names=column_names)
order_high_low = ['low', 'med', 'high', 'vhigh']
order_size = ['small', 'med', 'big']
order_safety = ['low', 'med', 'high']
order_doors = ['2', '3', '4', '5more']
order_persons = ['2', '4', 'more']
order_class = ['unacc', 'acc', 'good', 'vgood']
categories = [order_high_low, order_high_low, order_doors, order_persons, order_size, order_safety, order_class]
encoder = OrdinalEncoder(categories=categories)
df_encoded = pd.DataFrame(encoder.fit_transform(df), columns=column_names)

# 2. Prepare Features (X) and Target (y)
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

# 3. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. Build and Train the Decision Tree Classifier
# criterion='gini', max_depth=3
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
clf_gini.fit(X_train, y_train)

# 5. Predict on the Training set
y_train_pred = clf_gini.predict(X_train)

# 6. Calculate Training Accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)

print(f"Training Set Accuracy Score: {train_accuracy:.4f}")

Training Set Accuracy Score: 0.7974


training and test accuracy are similar 0.7974 and 0.7784 respectively, these metrics are close so there is no overfitting in the model.

In [31]:
# part e (confusion matrix)

from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
print(conf_matrix)

[[347  16   0   0]
 [ 58  57   0   0]
 [  0  21   0   0]
 [  0  20   0   0]]


In [35]:
# part f (visualizing decision tree)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn import tree # <-- Ensures 'tree' is defined
import numpy as np

# --- 1. Data Setup (Fixes NameError by defining variables) ---
column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
# Assuming 'car_evaluation.csv' is in the current directory
df = pd.read_csv("car_evaluation.csv", header=None, names=column_names)

# Ordinal Encoding Mappings
order_high_low = ['low', 'med', 'high', 'vhigh']
order_size = ['small', 'med', 'big']
order_safety = ['low', 'med', 'high']
order_doors = ['2', '3', '4', '5more']
order_persons = ['2', '4', 'more']
order_class = ['unacc', 'acc', 'good', 'vgood']

categories = [order_high_low, order_high_low, order_doors, order_persons, order_size, order_safety, order_class]
encoder = OrdinalEncoder(categories=categories)
df_encoded = pd.DataFrame(encoder.fit_transform(df), columns=column_names)

# Define X, y, feature_names, and class_names
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']
feature_names = X.columns.tolist() # <--- DEFINED HERE
class_names = ['unacc', 'acc', 'good', 'vgood']

# Split Data (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Train GINI model (clf_gini, from part c)
clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
clf_gini.fit(X_train, y_train)

# Train ENTROPY model (clf_entropy, from part h)
clf_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=42)
clf_entropy.fit(X_train, y_train)

# --- 2. Part (f): Gini Visualization ---
print("--- Part (f): Gini Visualization ---")

plt.figure(figsize=(15, 10))
tree.plot_tree(clf_gini,
               feature_names=feature_names,
               class_names=class_names,
               filled=True,
               rounded=True,
               fontsize=8)
plt.title("Decision Tree (Criterion: Gini, Max Depth: 3)")
plt.savefig("decision_tree_gini.png")
plt.close()
print("Visualization saved to decision_tree_gini.png")

# --- 3. Part (h): Entropy Visualization and Results ---
print("\n--- Part (h): Entropy Visualization and Results ---")

# Calculate metrics for the Entropy model
test_accuracy_entropy = accuracy_score(y_test, clf_entropy.predict(X_test))
train_accuracy_entropy = accuracy_score(y_train, clf_entropy.predict(X_train))
conf_matrix_entropy = confusion_matrix(y_test, clf_entropy.predict(X_test))

plt.figure(figsize=(15, 10))
tree.plot_tree(clf_entropy,
               feature_names=feature_names,
               class_names=class_names,
               filled=True,
               rounded=True,
               fontsize=8)
plt.title("Decision Tree (Criterion: Entropy, Max Depth: 3)")
plt.savefig("decision_tree_entropy.png")
plt.close()
print("Visualization saved to decision_tree_entropy.png")

print(f"Entropy Training Accuracy: {train_accuracy_entropy:.4f}")
print(f"Entropy Test Accuracy: {test_accuracy_entropy:.4f}")
print(f"Entropy Confusion Matrix:\n{conf_matrix_entropy}")


# --- 4. Part (i): Comparison ---
print("\n--- Part (i): Comparison ---")

test_accuracy_gini = accuracy_score(y_test, clf_gini.predict(X_test))
gini_structure = clf_gini.tree_.children_left.tolist()
entropy_structure = clf_entropy.tree_.children_left.tolist()
is_same_structure = (gini_structure == entropy_structure)

print(f"Gini Test Accuracy:    {test_accuracy_gini:.4f}")
print(f"Entropy Test Accuracy: {test_accuracy_entropy:.4f}")
print(f"Are the model structures the same? {is_same_structure}")

if is_same_structure:
    print("Conclusion: The trees are identical in structure and performance for max_depth=3.")
else:
    print("Conclusion: The trees differ in structure, compare their test accuracy for the final choice.")

--- Part (f): Gini Visualization ---
Visualization saved to decision_tree_gini.png

--- Part (h): Entropy Visualization and Results ---
Visualization saved to decision_tree_entropy.png
Entropy Training Accuracy: 0.7974
Entropy Test Accuracy: 0.7784
Entropy Confusion Matrix:
[[347  16   0   0]
 [ 58  57   0   0]
 [  0  21   0   0]
 [  0  20   0   0]]

--- Part (i): Comparison ---
Gini Test Accuracy:    0.7784
Entropy Test Accuracy: 0.7784
Are the model structures the same? True
Conclusion: The trees are identical in structure and performance for max_depth=3.


In [37]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
